In [1]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
    
#From my EEG package 
import run_expts
import constants


#Let me see as many results as I want to see
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 20)

In [2]:
#This path also has Wakeful data in it
load_path = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Execute New Experiments/Catch_22_features/'

data_types =  ['N1' , 'REM']
dtype = data_types[1]

X = pd.read_hdf(load_path + dtype +  '_c_22_feautures.h5', key='df', mode='r')
y = pd.read_hdf(load_path + dtype +  '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + dtype +  '_groups.h5', key='df', mode='r')

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )

In [3]:
# This code generates a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}

region_channel_dict = constants.region_to_channel_dict

for region, region_channels in region_channel_dict.items():
    region_features = []
    for channel in region_channels:
        region_features += [feature for feature in X_expt.columns if  feature.startswith(channel) ]
    regional_features_dict[region] = region_features

#Default subsets dictionary ---> run_expts.gen_subsets_dict()

#New subsets dictionary ---> regional_features_dict

In [4]:
t1 = time.time()
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier()} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
t2 = time.time()
t2 - t1

22.95887565612793

In [5]:
results_df

,random_state,y_true,Prefrontal_RF_y_preds,Frontal_RF_y_preds,Left Frontal_RF_y_preds,Right Frontal_RF_y_preds,Central_RF_y_preds,Left Central_RF_y_preds,Right Central_RF_y_preds,Left Temporal_RF_y_preds,Right Temporal_RF_y_preds,Parietal_RF_y_preds,Left Parietal_RF_y_preds,Right Parietal_RF_y_preds,Occipital_RF_y_preds
0,1,"[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 1, 0, 0, ...","[[0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [1, 1, 1, 0, ...","[[0, 0, 0, 0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 0, ...","[[0, 0, 0, 0, 1, 0, 1, 1, 1, 1], [1, 1, 0, 0, ...","[[0, 0, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, ...","[[0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, ...","[[0, 1, 0, 1, 0, 0, 0, 0, 0, 1], [1, 0, 0, 1, ...","[[1, 1, 1, 0, 1, 0, 0, 0, 1, 1], [1, 1, 0, 0, ...","[[0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 0, ...","[[1, 0, 1, 0, 0, 1, 0, 1, 1, 0], [0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, ..."
1,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, ...","[[1, 1, 0, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, ...","[[1, 1, 1, 0, 0, 1, 1, 1, 1, 0], [0, 0, 0, 1, ...","[[1, 1, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, ...","[[1, 1, 0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, ...","[[1, 1, 1, 0, 1, 0, 1, 0, 0, 0], [1, 0, 0, 0, ...","[[1, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, ...","[[1, 1, 1, 0, 1, 0, 0, 1, 1, 0], [1, 1, 0, 0, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 0, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, ...","[[1, 1, 0, 0, 1, 1, 0, 0, 0, 0], [1, 1, 0, 0, ...","[[1, 1, 0, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, ..."


In [7]:
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   
res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) 

Prefrontal_RF_acc        0.465556
Frontal_RF_acc           0.415556
Left Frontal_RF_acc      0.392222
Right Frontal_RF_acc     0.570000
Central_RF_acc           0.508889
Left Central_RF_acc      0.393333
Right Central_RF_acc     0.394444
Left Temporal_RF_acc     0.414444
Right Temporal_RF_acc    0.413333
Parietal_RF_acc          0.455556
Left Parietal_RF_acc     0.357778
Right Parietal_RF_acc    0.388889
Occipital_RF_acc         0.463333
dtype: float64

### REM Results

In [6]:
res_df

NameError: name 'res_df' is not defined

In [23]:
results_df
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   
res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) 

Prefrontal_RF_acc        0.505556
Frontal_RF_acc           0.414444
Left Frontal_RF_acc      0.392222
Right Frontal_RF_acc     0.595556
Central_RF_acc           0.478889
Left Central_RF_acc      0.384444
Right Central_RF_acc     0.485556
Left Temporal_RF_acc     0.485556
Right Temporal_RF_acc    0.371111
Parietal_RF_acc          0.413333
Left Parietal_RF_acc     0.371111
Right Parietal_RF_acc    0.513333
Occipital_RF_acc         0.422222
dtype: float64

### N1 Results below

In [13]:
res_df

,random_state,Prefrontal_RF_acc,Prefrontal_RF_std,Frontal_RF_acc,Frontal_RF_std,Left Frontal_RF_acc,Left Frontal_RF_std,Right Frontal_RF_acc,Right Frontal_RF_std,Central_RF_acc,...,Right Temporal_RF_acc,Right Temporal_RF_std,Parietal_RF_acc,Parietal_RF_std,Left Parietal_RF_acc,Left Parietal_RF_std,Right Parietal_RF_acc,Right Parietal_RF_std,Occipital_RF_acc,Occipital_RF_std
0,1,0.475,0.183712,0.50,0.136931,0.425,0.169558,0.55,0.127475,0.575,...,0.550,0.061237,0.650,0.215058,0.575,0.127475,0.450,0.150000,0.6,0.229129
1,2,0.550,0.244949,0.65,0.165831,0.525,0.122474,0.65,0.122474,0.675,...,0.625,0.158114,0.625,0.079057,0.650,0.122474,0.525,0.093541,0.6,0.145774


In [12]:
results_df
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   
res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) 

Prefrontal_RF_acc        0.5125
Frontal_RF_acc           0.5750
Left Frontal_RF_acc      0.4750
Right Frontal_RF_acc     0.6000
Central_RF_acc           0.6250
Left Central_RF_acc      0.6750
Right Central_RF_acc     0.6250
Left Temporal_RF_acc     0.6000
Right Temporal_RF_acc    0.5875
Parietal_RF_acc          0.6375
Left Parietal_RF_acc     0.6125
Right Parietal_RF_acc    0.4875
Occipital_RF_acc         0.6000
dtype: float64